In [1]:
import logging
import os
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
pd.options.display.max_columns = 999

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [3]:
# t_c = 300
# t_v = 300
# K = 2
# n = 1
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = GlobalExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [4]:
t_c = 100
t_v = 100
K = 2
n = 2
max_iter = 1
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = PcitExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-14 22:51:27] INFO      [PCIT] Start!
[2024-11-14 22:51:27] INFO      Attempt 1/2
[2024-11-14 22:51:27] INFO      Clustering: {'data/TSP/CEPS_benchmark/cluster_netgen/40.tsp': 0, 'data/TSP/CEPS_benchmark/grid/04.tsp': 1, 'data/TSP/CEPS_benchmark/grid/33.tsp': 0, 'data/TSP/CEPS_benchmark/rotation/45.tsp': 1, 'data/TSP/CEPS_benchmark/uniform_portgen/11.tsp': 0}
[2024-11-14 22:51:27] INFO      Solver 1/2
[2024-11-14 22:51:27] DEBUG     SMAC configuration, time: [100.]
[2024-11-14 22:51:27] DEBUG     executor start
[2024-11-14 22:51:34] DEBUG     data/TSP/CEPS_benchmark/cluster_netgen/40.tsp features calculated
[2024-11-14 22:51:35] DEBUG     data/TSP/CEPS_benchmark/grid/33.tsp features calculated
[2024-11-14 22:51:35] DEBUG     data/TSP/CEPS_benchmark/uniform_portgen/11.tsp features calculated
[2024-11-14 22:51:35] DEBUG     (0, 0) fn submitted
[2024-11-14 22:51:35] DEBUG     (1, 0) fn submitted
[2024-11-14 22:51:35] DEBUG     (2, 0) fn submitted
[2024-11-14 22:51:35] DEBUG     (0

In [ ]:
# t_c = 30
# t_v = 30
# K = 2
# n = 3
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = ParhydraExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
remaining_time = np.ones(shape=(K, )) * np.inf
best_portfolio.evaluate(test_instances, remaining_time, "test")

In [5]:
from src.database import db_connect
conn = db_connect()

In [ ]:
import sqlite3
conn = sqlite3.connect("database/2024_11_12_23_10_13.db")

In [6]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,2220588739511820280,52,1,YES,ALPHA,7,QUADRANT,NO,YES,1,QUICK-BORUVKA,0.582020,3,0,954863081,10,3,3,1,8,YES,YES,6,NO
1,2051815471081033189,45,0,YES,DELAUNAY,10,QUADRANT,YES,YES,3,SIERPINSKI,0.842342,3,5,1010150627,9,6,3,1,26,YES,NO,4,NO


In [7]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,centroid_centroid_x,centroid_centroid_y,centroid_dist_min,centroid_dist_median,centroid_dist_mean,centroid_dist_max,centroid_dist_sd,centroid_dist_span,centroid_dist_coef_of_var,cluster_01pct_number_of_clusters,cluster_01pct_mean_distance_to_centroid,cluster_05pct_number_of_clusters,cluster_05pct_mean_distance_to_centroid,cluster_10pct_number_of_clusters,cluster_10pct_mean_distance_to_centroid,bounding_box_10_ratio_of_cities_outside_box,bounding_box_20_ratio_of_cities_outside_box,bounding_box_30_ratio_of_cities_outside_box,chull_area,chull_points_on_hull,distance_distances_shorter_mean_distance,distance_distinct_distances,distance_mode_frequency,distance_mode_quantity,distance_mode_mean,distance_mean_tour_length,distance_sum_of_lowest_edge_values,distance_min,distance_median,distance_mean,distance_max,distance_sd,distance_span,distance_coef_of_var,modes_number,mst_depth_min,mst_depth_median,mst_depth_mean,mst_depth_max,mst_depth_sd,mst_depth_span,mst_depth_coef_of_var,mst_dists_min,mst_dists_median,mst_dists_mean,mst_dists_max,mst_dists_sd,mst_dists_span,mst_dists_coef_of_var,mst_dists_sum,nnds_min,nnds_median,nnds_mean,nnds_max,nnds_sd,nnds_span,nnds_coef_of_var,num_nodes,cost_matrix_avg,cost_matrix_std,cost_matrix_skew,stdTime,mst_length,mst_length_avg,mst_length_std,mst_length_skew,mst_degree_avg,mst_degree_std,mst_degree_skew,mstTime,cluster_distance_avg,cluster_distance_std,cluster_distance_skew,clusterTime,tour_const_heu_avg,tour_const_std,tour_const_skew,ls_impov_per_step_avg,ls_impov_per_step_std,ls_impov_per_step_skew,ls_steps_2lm_avg,ls_steps_2lm_std,ls_steps_2lm_skew,ls_maxdist_avg,ls_maxdist_std,ls_maxdist_skew,ls_bestsol_avg,ls_bestsol_std,ls_bestsol_skew,ls_backbone_avg,ls_backbone_std,ls_backbone_skew,lpTime,bc_improv_per_cut_avg,bc_improv_per_cut_std,bc_improv_per_cut_skew,bc_upper_lower_ratio,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,0.001897,1.430042,1.491193,3.140406,0.927441,3.138509,0.621946,0.471871,0.487806,0.040004,0.369238,0.381506,0.706402,0.151182,0.666398,0.396278,23.0,0.024919,7.0,0.047305,5.0,0.096363,0.30250,0.45500,0.85500,0.739118,0.02250,0.458728,1.0,800.0,0.000002,0.0,818.771792,4.210875,0.0,0.533828,0.511093,1.350294,0.274833,1.350294,0.537736,4.0,1.0,5.0,13.24125,80.0,18.549748,79.0,1.400906,0.000219,0.008216,0.011043,0.339508,0.016342,0.339289,1.479878,0.000027,0.000219,0.005963,0.008050,0.070918,0.007696,0.070699,0.956124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,data/TSP/CEPS_benchmark/grid/04.tsp,0.001978,1.651404,1.641448,3.141593,0.934762,3.139615,0.569474,0.517514,0.488799,0.010197,0.405103,0.386497,0.717120,0.138593,0.706924,0.358587,1.0,0.384896,1.0,0.386497,1.0,0.386497,0.37375,0.66125,0.85125,0.984655,0.01875,0.510500,1.0,800.0,0.000002,0.0,840.219791,16.411909,0.0,0.516829,0.524481,1.381468,0.249095,1.381468,0.474936,2.0,1.0,5.0,13.65625,83.0,19.182819,82.0,1.404692,0.001058,0.024411,0.024615,0.067842,0.010731,0.066784,0.435948,0.000059,0.001058,0.018953,0.019898,0.067842,0.010276,0.066784,0.516433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,2220588739511820280,100.00,10.00,,2024-11-14 19:53:15
1,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,2051815471081033189,100.00,10.00,,2024-11-14 19:53:28
2,data/TSP/CEPS_benchmark/grid/04.tsp,2220588739511820280,100.00,10.00,,2024-11-14 19:53:28
3,data/TSP/CEPS_benchmark/grid/04.tsp,2051815471081033189,8.89,8.89,,2024-11-14 19:53:28


In [ ]:
df.loc[df["comment"] == "configuration"]

In [ ]:
_ = df.loc[df["comment"] == "test"].pivot_table(index="instance_id", columns="solver_id", values="cost")
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

In [ ]:
_["min"].describe()

In [ ]:
(_["min"] > 90).value_counts()

In [ ]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

In [ ]:
_.sort_values(by="min")

In [ ]:
config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
solver = TSP_LKH_Solver(config=config)

instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index